In [1]:
import pandas as pd
import json
import anthropic

## Preparation

In [2]:
client = anthropic.Anthropic(
    api_key='<paste_claude_key_here>',
)

**read data**

In [14]:
prompts = pd.read_csv('data/prompts.csv') # read settings
articles = pd.read_csv('data/evaluation_texts.csv') # read articles
criteria = pd.read_csv('data/criteria.csv') # read criteria

**define iteration variables**

In [15]:
techniques = prompts['key'].tolist()
criteria = criteria['kriterien'].tolist()
models = ['claude-3-5-sonnet-20240620']

In [ ]:
# select subset for demonstration purposes
articles = articles.iloc[0:2]
techniques = techniques[0:2]
criteria = criteria[0:2]

## Start Loop

In [ ]:
all_results = {}

for model in models:
    
    print('\n#### - MODEL: ', model, ' - ####\n')
    
    # init list for model 
    results_per_model = {}

    # iterate over all techniques
    for technic in techniques:

        print('\nNEXT PROMPT STYLE:', technic, '\n')

        # get criteria right
        krits = criteria if 'each' in technic else ['\n'.join(f'{i+1} {criteria[i]}' for i in range(len(criteria)))]

        # get instruction right
        instruction = prompts[prompts['key'] == technic]['value'].tolist()

        # init predictions_list that entails all articles
        predictions = {}

        # iterate over all documents
        for row in articles.itertuples():

            print('  article:', row.id)
            
            # init prediction_per_crit list that entails the predictions for one single document
            prediction_per_crit = {}

            # iterate over all criteria
            for idx, krit in enumerate(krits):

                print('   crit:', krit)

                # create complete promt
                prompt = f'{instruction}\n\n#ARTIKEL:\n{row.text}\n\n#KRIT.:\n{krit}'

                # mimic three runs (claude does not offer a parameter 'n')
                runs = []
                for i in [0,1,2]:

                    chat_completion = client.messages.create(
                        model=model,
                        system="Du bist strenger wissenschaftlicher Gutachter von wissenschaftsjournalistischen Texten.",
                        messages=[
                            {"role": "user", "content": prompt},
                        ],
                        max_tokens=1024,
                        temperature=0.1
                    )

                    runs.append(chat_completion.content[0].text)

                # append predictions_per_crit list (one article - all predictions)
                prediction_per_crit['prompt'+str(idx)] = runs

            # append predictions list (all articles - same prompting)
            predictions[row.id] = prediction_per_crit

        # append results_per_model list - data from all runs for selected model
        results_per_model[technic] = predictions

    # append all_results list - all data
    all_results[model] = results_per_model

# return
# all_results

In [62]:
# save locally
with open('Claude_evaluations.json', 'w', encoding='utf-8') as outfile: 
    json.dump(all_results, outfile, indent=5, ensure_ascii=False)